In [1]:
import m2mcluster as m2m

In [2]:
from amuse.lab import *
from amuse.units import nbody_system,units

import matplotlib.pyplot as plt
import numpy as np


In [3]:
from amuse.community.hermite.interface import Hermite


In [4]:
import time

In [5]:
N=1000
Mcluster = 1000 | units.MSun
Rcluster = 9. |units.parsec

mmin=0.1 | units.MSun
mmax=1.4 | units.MSun
mtot = Mcluster

niterations=10
snapfreq=1

timing=False
doplots=True

In [6]:
ocluster,oconverter=m2m.setup_star_cluster(N=N, Mcluster=Mcluster, Rcluster = Rcluster, W0=7.,imf='kroupa',mmin=mmin, mmax=mmax)


In [7]:
#Setup a star cluster for artificial observation
ocluster,oconverter=m2m.setup_star_cluster(N=N, Mcluster = Mcluster, Rcluster = Rcluster, W0=7.)

#Measure artifical cluster's density profile asssuming an equal number of stars per bin
orlower,orad,orupper,orho=m2m.density(ocluster,bins=True,bintype='fix')
orlower[0]=0.
print(orho)

[  2.30324539e+00   4.53969792e-01   1.93310438e-01   5.84621227e-02
   3.93510052e-02   1.82782508e-02   1.11635764e-02   5.68452017e-03
   4.73364116e-03   1.46337721e-03   1.27617243e-03   1.10168649e-03
   7.92101444e-04   1.87779610e-04   1.07146911e-04   1.11369048e-04
   2.57822910e-05   1.35358870e-04   0.00000000e+00   3.26661508e-05]


In [8]:
while np.sum(orho==0.)!=0:
    ocluster,oconverter=m2m.setup_star_cluster(N=N, Mcluster = Mcluster, Rcluster = Rcluster, W0=7.)
    orlower,orad,orupper,orho=m2m.density(ocluster,bins=True,bintype='fix')
    orlower[0]=0.


In [9]:
sigv=m2m.velocity_dispersion(ocluster,rlower=orlower,rmid=orad,rupper=orupper)


In [10]:
#Initialize an M2M Star cluster
#Specify number of iterations to run algorithm for
#Specify number of workers to be used by Nbody code
cluster=m2m.starcluster(number_of_iterations=niterations,number_of_workers=1,debug=False)

In [11]:
#Add the "observed" cluster density profile as an observable
cluster.add_observable(orlower,orad,orupper,orho,'density',ndim=3)
cluster.add_observable(orlower,orad,orupper,sigv,'velocity',ndim=3)

In [12]:
#Initialize a model star cluster will an initial guess as the observed cluster's properties
cluster.initialize_star_cluster(N=N, Mcluster = Mcluster, Rcluster = Rcluster, softening = 0.01 | units.parsec, W0=1.,imf='single',mmin=1. | units.MSun)

(<amuse.datamodel.particles.Particles at 0x136e8d910>,
 <amuse.units.nbody_system.nbody_to_si at 0x136f5a9a0>)

In [13]:
cluster.writeout()
cluster.snapout()

In [14]:
if doplots: 
    #Plot initial positions
    cluster.xy_plot(filename='xyplot0.png')
    #Compare initial density profiles
    cluster.rho_prof(filename='rhoplot0.png')
    #Compare initial velocity dispersion profiles
    if doplots: cluster.sigv_prof(filename='sigplot0.png')

In [15]:
#Compare initial density profiles
if doplots: cluster.rho_prof(filename='rhoplot0.png')


In [16]:
if doplots: cluster.sigv_prof(filename='sigplot0.png')


In [17]:
#Exectute the made to measure algorithm
for i in range(0,cluster.number_of_iterations):
    if timing: dttime=time.time()
    #Initialize a new N-body simulation ever time step. 
    #In this example I use 1% of the cluster's dynamical time for the integration timeste
    cluster.initialize_gravity_code('Hermite', dt=0.01*cluster.tdyn, theta=0.6)
    
    if timing: print('initialize_gravity_code',time.time()-dttime)
    if timing: dttime=time.time()
    
    #Evolve the model cluster forward for 10% of its dynamical time
    cluster.evolve(tend=0.1*cluster.tdyn)
    
    if timing: print('evolve',time.time()-dttime)
    if timing: dttime=time.time()
    
    #Run the M2M algorithm, which will adjust all of the stellar masses based on kernel function
    cluster.evaluate(kernel=None,epsilon=1.,mu=1.,alpha=1.)
    
    if timing: print('evaluate',time.time()-dttime)
    if timing: dttime=time.time()
    
    #Write profiles and chi^2 to outfile
    cluster.writeout()
    
    if timing: print('writeout',time.time()-dttime)
    if timing and doplots: dttime=time.time()
        
    #Compare the new model density profile to the observed one
    if doplots: cluster.rho_prof(filename='%s.png' % str(i).zfill(5))
    
    if timing and doplots: print('rho_prof',time.time()-dttime)
    if timing: dttime=time.time()
        
    #Centre the star cluster and find determine Nbody conversion scales for next integration
    cluster.reinitialize_star_cluster(mmin= mmin, mmax=mmax, mtot=mtot)

    if timing: print('reinitialize_star_cluster',time.time()-dttime)
    if timing: dttime=time.time()
    
    if cluster.niteration>=snapfreq:
        cluster.snapout()
        snapfreq+=snapfreq
        
        if timing: print('snapout',time.time()-dttime)
        
cluster.outfile.close()

initialize_gravity_code 0.507849931716919
TIME UNITS:  1.27286991561 Myr 1000 1000.0 8.99984238189
evolve 7.2470221519470215
evaluate 0.06203484535217285
writeout 0.004591941833496094
rho_prof 0.48241400718688965
reinitialize_star_cluster 0.12503600120544434
snapout 0.009593725204467773
initialize_gravity_code 0.23082494735717773
TIME UNITS:  1.26422108541 Myr 1000 1000.0 8.95918520394
evolve 9.037790060043335
evaluate 0.06530094146728516
writeout 0.004353761672973633
rho_prof 0.41956377029418945
reinitialize_star_cluster 0.10241889953613281
snapout 0.009097099304199219
initialize_gravity_code 0.19863295555114746
TIME UNITS:  1.26226306069 Myr 1000 1000.0 8.94993216129
evolve 6.650537967681885
evaluate 0.06276822090148926
writeout 0.00433802604675293
rho_prof 0.35042381286621094
reinitialize_star_cluster 0.10130000114440918
initialize_gravity_code 0.19402432441711426
TIME UNITS:  1.25950467058 Myr 1000 1000.0 8.93688870764
evolve 4.5284950733184814
evaluate 0.06274986267089844
writeout

In [18]:
if doplots: 
    #Plot initial positions
    cluster.xy_plot(filename='xyplotf.png')
    #Compare initial density profiles
    cluster.rho_prof(filename='rhoplotf.png')
    #Compare initial velocity dispersion profiles
    if doplots: cluster.sigv_prof(filename='sigplotf.png')